 # Import Required Libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import os


# Preprocessing and Data Loaders

In [2]:
# Common Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Paths to data directories
train_dir = r'C:\Users\harsh\OneDrive\Desktop\archive (1)\new\train'
test_dir = r'C:\Users\harsh\OneDrive\Desktop\archive (1)\new\validation'

# Datasets
train_data = datasets.ImageFolder(train_dir, transform=transform)
test_data = datasets.ImageFolder(test_dir, transform=transform)

# Binary classification: Map labels for binary
binary_train_data = [(img, 0 if label == 0 else 1) for img, label in train_data]
binary_test_data = [(img, 0 if label == 0 else 1) for img, label in test_data]

# Loaders
binary_train_loader = DataLoader(binary_train_data, batch_size=32, shuffle=True)
binary_test_loader = DataLoader(binary_test_data, batch_size=32, shuffle=False)

multi_train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
multi_test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

class_names = train_data.classes
print(f"Classes: {class_names}")


Classes: ['clean', 'dust', 'electrical', 'physical', 'snow']


# Define Models
## Binary Classification Model (ResNet)
## Multiple Classification Model (EfficientNet)

In [3]:
# Binary Classification Model using ResNet
class BinaryClassifier(nn.Module):
    def __init__(self):
        super(BinaryClassifier, self).__init__()
        self.base_model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        self.base_model.fc = nn.Linear(self.base_model.fc.in_features, 2)

    def forward(self, x):
        return self.base_model(x)

# Multi-class Classification Model using EfficientNet
class MultiClassifier(nn.Module):
    def __init__(self):
        super(MultiClassifier, self).__init__()
        self.base_model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)
        self.base_model.classifier[1] = nn.Linear(self.base_model.classifier[1].in_features, 5)

    def forward(self, x):
        return self.base_model(x)


# Define Training and Evaluation Functions

In [4]:
def train(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0.0
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    accuracy = correct / total
    return total_loss / len(loader), accuracy


# Training Both Models

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Binary Classification Training
binary_model = BinaryClassifier().to(device)
binary_criterion = nn.CrossEntropyLoss()
binary_optimizer = optim.Adam(binary_model.parameters(), lr=0.001)

# Multi-class Classification Training
multi_model = MultiClassifier().to(device)
multi_criterion = nn.CrossEntropyLoss()
multi_optimizer = optim.Adam(multi_model.parameters(), lr=0.001)

# Training Loop
epochs = 10
for epoch in range(epochs):
    # Train Binary Model
    binary_loss = train(binary_model, binary_train_loader, binary_criterion, binary_optimizer, device)
    binary_val_loss, binary_accuracy = evaluate(binary_model, binary_test_loader, binary_criterion, device)
    
    # Train Multi-class Model
    multi_loss = train(multi_model, multi_train_loader, multi_criterion, multi_optimizer, device)
    multi_val_loss, multi_accuracy = evaluate(multi_model, multi_test_loader, multi_criterion, device)
    
    print(f"Epoch {epoch+1}/{epochs}")
    print(f"Binary - Loss: {binary_loss:.4f}, Val Loss: {binary_val_loss:.4f}, Accuracy: {binary_accuracy:.4f}")
    print(f"Multi - Loss: {multi_loss:.4f}, Val Loss: {multi_val_loss:.4f}, Accuracy: {multi_accuracy:.4f}")


Epoch 1/10
Binary - Loss: 0.6742, Val Loss: 1.3042, Accuracy: 0.5763
Multi - Loss: 0.9236, Val Loss: 2.2169, Accuracy: 0.5678
Epoch 2/10
Binary - Loss: 0.3738, Val Loss: 0.4104, Accuracy: 0.8390
Multi - Loss: 0.3733, Val Loss: 0.3744, Accuracy: 0.8559
Epoch 3/10
Binary - Loss: 0.2188, Val Loss: 0.3518, Accuracy: 0.9153
Multi - Loss: 0.1639, Val Loss: 0.4458, Accuracy: 0.8390
Epoch 4/10
Binary - Loss: 0.2321, Val Loss: 0.5527, Accuracy: 0.8136
Multi - Loss: 0.0995, Val Loss: 0.3414, Accuracy: 0.8814
Epoch 5/10
Binary - Loss: 0.1510, Val Loss: 0.9310, Accuracy: 0.7797
Multi - Loss: 0.1124, Val Loss: 0.5717, Accuracy: 0.8220
Epoch 6/10
Binary - Loss: 0.1312, Val Loss: 0.6184, Accuracy: 0.8559
Multi - Loss: 0.1243, Val Loss: 0.5122, Accuracy: 0.8475
Epoch 7/10
Binary - Loss: 0.1810, Val Loss: 1.7844, Accuracy: 0.6864
Multi - Loss: 0.1098, Val Loss: 0.5283, Accuracy: 0.8475
Epoch 8/10
Binary - Loss: 0.2029, Val Loss: 0.2894, Accuracy: 0.8983
Multi - Loss: 0.0796, Val Loss: 0.6262, Accuracy:

# Classify Images

In [7]:
from PIL import Image  # Import the Image class

def classify_image(binary_model, multi_model, image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])
    image = Image.open(image_path).convert("RGB")  # Use PIL.Image to open the image
    image = transform(image).unsqueeze(0).to(device)

    binary_model.eval()
    multi_model.eval()

    with torch.no_grad():
        # Binary classification
        binary_output = binary_model(image)
        is_unclean = torch.argmax(binary_output, dim=1).item()

        if is_unclean == 1:
            # Multi-class classification
            multi_output = multi_model(image)
            subcategory = torch.argmax(multi_output, dim=1).item()
            subcategory_name = class_names[subcategory]
            print(f"The panel is classified as 'unclean' with subcategory: {subcategory_name}")
        else:
            print("The panel is classified as 'clean'.")

image_path = r'C:\Users\harsh\OneDrive\Desktop\design1\okay.jpg'
classify_image(binary_model, multi_model, image_path)


The panel is classified as 'unclean' with subcategory: snow
